In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

In [2]:
# 제로샷 테스트를 하기 위해 모델을 다운받고, 인퍼런스를 실행합니다. 
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = ""
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": 'Task:라틴 아메리카/카리브해 지역의 인구가 783(7.5%)가 될 때, 아시아의 인구는 얼마가 될까요?\nSQL Table:CREATE TABLE table_22767 (\n    "Year" real,\n    "World" real,\n    "Asia" text,\n    "Africa" text,\n    "Europe" text,\n    "Latin America/Caribbean" text,\n    "Northern America" text,\n    "Oceania" text\n)\nQuery:'}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
# model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# generated_ids = model.generate(
#     **model_inputs,
#     max_new_tokens=512
# )
# generated_ids = [
#     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
# ]

# response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


/root/anaconda3/envs/text_to_sql/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.15s/it]


# 제로샷 테스트 

### 정답 
SELECT "Asia" FROM table_22767 WHERE "Latin America/Caribbean" = \'783 (7.5%)\'

### 생성 결과
SELECT (783 / 0.075) AS World_Population, (0.15 * (783 / 0.075)) AS Asia_Population;

잘못된 결과를 생성하고 있고 우리가 원하지 않는 문장들이 포함되어 있습니다.   
Fine-Tuning을 통해 이러한 문제를 해결해보겠습니다. 

In [4]:
import warnings
warnings.filterwarnings("ignore")

import trl
import torch
import datasets
import transformers

import pandas as pd
from random import randint
from datasets import Dataset, load_dataset

from trl import SFTTrainer, setup_chat_format
from peft import LoraConfig, AutoPeftModelForCausalLM

import wandb
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline)

from huggingface_hub import login

import os
import json
from openai import OpenAI

In [5]:
print(f"PyTorch version       : {torch.__version__}")
print(f"Transformers version  : {transformers.__version__}")
print(f"TRL version           : {trl.__version__}")
print(f"CUDA available        : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version      : {torch.version.cuda}")

PyTorch version       : 2.5.1
Transformers version  : 4.47.1
TRL version           : 0.13.0
CUDA available        : True
CUDA version      : 12.1


In [ ]:
login(
  token="Huggingface_Token", # 여기에 토큰 추가 
)

In [7]:
# dataset = datasets.load_dataset("Clinton/text-to-sql-v1")
dataset = datasets.load_dataset("daje/kotext-to-sql-v1")
dataset     

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response', 'source', 'text', 'ko_instruction'],
        num_rows: 262208
    })
})

In [8]:
def add_length_column(dataset):
    df = dataset.to_pandas()
    df["total_length"] = 0
    for column_name in ["ko_instruction", "input", "response"]:
        num_words = df[column_name].astype(str).str.split().apply(len)
        df["total_length"] += num_words

    return df

df = add_length_column(dataset["train"])

def filter_by_total_length(df, difficulty, number_of_samples, random_state=8888):  # random_state 추가
    if difficulty == "easy":
        return df[df["total_length"].between(10, 100)].sample(n=number_of_samples, random_state=random_state)  # iloc 대신 sample 사용
    elif difficulty == "moderate":
        return df[df["total_length"].between(101, 300)].sample(n=number_of_samples, random_state=random_state)
    elif difficulty == "difficult":
        return df[df["total_length"].between(301, 1000)].sample(n=number_of_samples, random_state=random_state)

print(max(df["total_length"].to_list()), min(df["total_length"].to_list()))

910 13


In [10]:
# trl docs에 보면 이와 같은 방식으로 SFT Trainer용 데이터를 만들 수 있습니다.
# docs에서는 eos_token을 별도로 추가하라는 안내는 없지만, 저자는 습관적으로 eos_token을 붙혀줍니다.
def get_chat_format(element):
    system_prompt = (
        "You are a helpful programmer assistant that excels at SQL. "
        "Below are sql tables schemas paired with instruction that describes a task. "
        "Using valid SQLite, write a response that appropriately completes the request for the provided tables."
    )
    user_prompt = "### instruction:{ko_instruction} ### Input:{input} ### response:"
    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt.format_map(element)},
            {"role": "assistant", "content": element["response"]+tokenizer.eos_token},
        ]
    }

tokenizer.padding_side = 'right'                      

def tokenize(element):
    # 참고 코드에서처럼, apply_chat_template 사용
    formatted = tokenizer.apply_chat_template(
        get_chat_format(element)["messages"],  # get_chat_format 반환 형식을 맞춰줍니다.
        tokenize=False
    )
    outputs = tokenizer(formatted)
    return {
        "input_ids": outputs["input_ids"],
        "attention_mask": outputs["attention_mask"],
    }
    
# 데이터를 일괄적으로 대화형식으로 변경합니다.

easy = filter_by_total_length(df, "easy", 100)
medium = filter_by_total_length(df, "moderate", 100)
hard = filter_by_total_length(df, "difficult", 100)
dataset = pd.concat([easy, medium, hard])
dataset = dataset.sample(frac=1, random_state=8888)  # random_state 추가
dataset = Dataset.from_pandas(dataset)
easy.shape, medium.shape, hard.shape, dataset.shape

temp_dataset = dataset.map(get_chat_format, remove_columns=dataset.features, batched=False)

# train과 test 데이터를 0.9와 0.1로 분할합니다.
temp_dataset = temp_dataset.train_test_split(test_size=0.05, seed=42)

temp = tokenizer.apply_chat_template(temp_dataset["train"][0]["messages"], tokenizer=False)
print(tokenizer.decode(temp))

Map: 100%|██████████| 300/300 [00:00<00:00, 7997.73 examples/s]

<|im_start|>system
You are a helpful programmer assistant that excels at SQL. Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.<|im_end|>
<|im_start|>user
### instruction:매일 오후 5시에 나가도 되나요, 370번과 489번을 타면? ### Input:CREATE TABLE course_prerequisite (
    pre_course_id int,
    course_id int
)

CREATE TABLE area (
    course_id int,
    area varchar
)

CREATE TABLE offering_instructor (
    offering_instructor_id int,
    offering_id int,
    instructor_id int
)

CREATE TABLE student (
    student_id int,
    lastname varchar,
    firstname varchar,
    program_id int,
    declare_major varchar,
    total_credit int,
    total_gpa float,
    entered_as varchar,
    admit_term int,
    predicted_graduation_semester int,
    degree varchar,
    minor varchar,
    internship varchar
)

CREATE TABLE program (
    program_id int,
    name varchar,
    college varch

In [9]:
# 일부 데이터만 샘플링하고 싶은 경우 
# easy = filter_by_total_length(df, "easy", 10000)
# medium = filter_by_total_length(df, "moderate", 10000)
# hard = filter_by_total_length(df, "difficult", 2000)
# dataset = pd.concat([easy, medium, hard])
# dataset = dataset.sample(frac=1, random_state=8888)  # random_state 추가
# dataset = Dataset.from_pandas(dataset)
# easy.shape, medium.shape, hard.shape, dataset.shape

# 전체 데이터로 학습을 할 경우 
dataset = Dataset.from_pandas(df)

In [10]:
# trl docs에 보면 이와 같은 방식으로 SFT Trainer용 데이터를 만들 수 있습니다.
# docs에서는 eos_token을 별도로 추가하라는 안내는 없지만, 저자는 습관적으로 eos_token을 붙혀줍니다.
def get_chat_format(element):
    system_prompt = (
        "You are a helpful programmer assistant that excels at SQL. "
        "Below are sql tables schemas paired with instruction that describes a task. "
        "Using valid SQLite, write a response that appropriately completes the request for the provided tables."
    )
    user_prompt = "### instruction:{ko_instruction} ### Input:{input} ### response:"
    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt.format_map(element)},
            {"role": "assistant", "content": element["response"]+tokenizer.eos_token},  # 여기 닫는 괄호 추가
        ]
    }

tokenizer.padding_side = 'right'                      

def apply_chat_format(element):
    """
    1) get_chat_format(element)를 호출해서
       messages를 생성한 뒤, Dataset에 저장할 dict로 반환합니다.
    """
    chat_format = get_chat_format(element)  # get_chat_format은 원본 코드 그대로 사용
    return {
        "messages": chat_format["messages"]
    }

def tokenize_messages(element):
    """
    2) apply_chat_template + tokenizer(...)를 통해
       input_ids와 attention_mask를 만들어 반환합니다.
    """
    # 위 단계에서 "messages"가 이미 Dataset에 들어가있다고 가정
    formatted = tokenizer.apply_chat_template(
        element["messages"],  # messages 리스트
        tokenize=False
    )
    outputs = tokenizer(formatted)
    return {
        "input_ids": outputs["input_ids"],
        "attention_mask": outputs["attention_mask"]
    }

# train과 test 데이터를 0.9와 0.1로 분할합니다.
dataset = dataset.map(
    apply_chat_format,
    batched=False,
    remove_columns=dataset.features,  # 원하시면 제거
)
dataset = dataset.train_test_split(test_size=0.02)
# dataset["train"].to_json("train_dataset.json", orient="records")
# dataset["test"].to_json("test_dataset.json", orient="records")

dataset = dataset.map(
    tokenize_messages,
    batched=False,
    remove_columns=["messages"],  # 이제 messages를 더이상 쓰지 않는다면 제거
)

Map: 100%|██████████| 5245/5245 [00:03<00:00, 1346.73 examples/s]


In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 256963
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 5245
    })
})

In [ ]:
from trl import DataCollatorForCompletionOnlyLM
response_template = "<|im_start|>assistant\n"
# 만약 실제로 줄바꿈까지 포함되어 있다면 \n까지 넣어줘야 합니다.
# response_template = "<|start_header_id|>assistant<|end_header_id|>\n"

response_template_ids = tokenizer.encode(response_template, add_special_tokens=False)
collator = DataCollatorForCompletionOnlyLM(
    response_template_ids,
    tokenizer=tokenizer
)


In [13]:
peft_config = LoraConfig(
        lora_alpha=128,                            
        lora_dropout=0.05,                         # Lora 학습 때 사용할 dropout 확률을 지정합니다. 드롭아웃 확률은 과적합 방지를 위해 학습 중 무작위로 일부 뉴런을 비활성화하는 비율을 지정합니다.
        r=256,                                     # Lora의 저차원 공간의 랭크를 지정합니다. 랭크가 높을수록 모델의 표현력이 증가하지만, 계산 비용도 증가합니다.
        bias="none",                               # Lora 적용 시 바이어스를 사용할지 여부를 설정합니다. "none"으로 설정하면 바이어스를 사용하지 않습니다.
        target_modules=["q_proj", "o_proj",        # Lora를 적용할 모델의 모듈 리스트입니다.
                        "k_proj", "v_proj"
                        "up_proj", "down_proj",
                        "gate_proj",
                        ],
        task_type="CAUSAL_LM",                     # 미세 조정 작업 유형을 CAUSAL_LM으로 지정하여 언어 모델링 작업을 수행합니다.
)


args = TrainingArguments(
    output_dir="Qwen2.5-260000_ko", # 모델 저장 및 허브 업로드를 위한 디렉토리 지정 합니다.
    num_train_epochs=1,                   # number of training epochs
    # max_steps=100,                          # 100스텝 동안 훈련 수행합니다.
    per_device_train_batch_size=1,          # 배치 사이즈 설정 합니다.
    gradient_accumulation_steps=2,          # 4스텝마다 역전파 및 가중치 업데이트합니다.
    gradient_checkpointing=False,            # 메모리 절약을 위해 그래디언트 체크포인팅 사용합니다.
    optim="adamw_torch_fused",              # 메모리 효율화할 수 있는 fused AdamW 옵티마이저 사용합니다.
    logging_steps=5000,                       # 10스텝마다 로그 기록합니다.
    save_strategy="steps",                  # 매 에폭마다 체크포인트 저장합니다.
    learning_rate=5e-5,                     # 학습률 2e-4로 설정 (QLoRA 논문 기반)합니다.
    bf16=True,                              # 정밀도 설정으로 학습 속도 향상합니다.
    tf32=True,
    max_grad_norm=0.3,                      # 그래디언트 클리핑 값 0.3으로 설정합니다.
    warmup_ratio=0.03,                      # 워밍업 비율 0.03으로 설정 (QLoRA 논문 기반)합니다.
    lr_scheduler_type="constant",           # 일정한 학습률 스케줄러 사용합니다.
    push_to_hub=False,                       # 훈련된 모델을 Hugging Face Hub에 업로드합니다.
    report_to="wandb",                      # wandb로 매트릭 관찰합니다.
)


trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    tokenizer=tokenizer,
    data_collator=collator,
)


In [ ]:
# trainer를 학습합니다.
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: daje0601. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss


In [19]:
# 메모리 초기화
del model
del trainer
torch.cuda.empty_cache()

In [3]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8001/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [4]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="/root/workspace/text_to_sql/test_dataset.json", split="train")
eval_dataset

/root/anaconda3/envs/text_to_sql/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['messages'],
    num_rows: 5245
})

In [3]:
eval_dataset[0]["messages"][:2]

[{'content': 'You are a helpful programmer assistant that excels at SQL. Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.',
  'role': 'system'},
 {'content': '### instruction:승리가 63이고 패배가 76보다 작은 경우, 승률은 얼마인가요? ### Input:CREATE TABLE table_32202 (\n    "Year" real,\n    "Team Name" text,\n    "Wins" real,\n    "Losses" real,\n    "W-L %" text,\n    "Manager" text\n) ### response:',
  'role': 'user'}]

In [4]:
idx = 0
sql_chat_completion = client.chat.completions.create(
    model="lora_adapter1",
    messages=eval_dataset[idx]["messages"][:2],
    temperature=0.1,
    max_tokens=500,
)

In [5]:
eval_dataset[idx]["messages"][2]

{'content': 'SELECT "W-L %" FROM table_32202 WHERE "Wins" = \'63\' AND "Losses" < \'76\'<|im_end|>',
 'role': 'assistant'}

In [6]:
print(sql_chat_completion.choices[0].message.content)

SELECT "W-L %" FROM table_32202 WHERE "Wins" = '63' AND "Losses" < '76'


In [ ]:
# 이런 경우 때문에 OpenAI 채점이 필요합니다. 
eval_dataset[idx]["messages"][2] == sql_chat_completion.choices[0].message.content

False

In [5]:
from tqdm.auto import tqdm 
result = [] 
for idx in tqdm(range(len(eval_dataset))):
    sql_chat_completion = client.chat.completions.create(
    model="lora_adapter1",
    messages=eval_dataset[idx]["messages"][:2],
    temperature=0.1,
    max_tokens=500,
    )
    result.append(
        (
            eval_dataset[idx]["messages"][:2],
            eval_dataset[idx]["messages"][2]["content"], 
            sql_chat_completion.choices[0].message.content
        )
    )


100%|██████████| 5245/5245 [43:57<00:00,  1.99it/s]  


In [6]:
import json 
from datetime import datetime

# 실수로 다시 실행해도 파일이 사라지지 않도록 하기 위해 timestamp를 사용하는걸 습관화하시는게 좋습니다. 
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"/root/workspace/text_to_sql/inference_output_1차_qwen2.5_{timestamp}.json"
 # 저장할 파일 이름
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

print(f"결과가 {output_file} 파일에 저장되었습니다.")

결과가 /root/workspace/text_to_sql/inference_output_1차_qwen2.5_20250111_160123.json 파일에 저장되었습니다.


In [1]:
import json 
output_file = "/root/workspace/text_to_sql/inference_output_1차_qwen2.5_20250111_160123.json"
with open(output_file, "r") as file:
    result = json.load(file)
result[0]

[[{'content': 'You are a helpful programmer assistant that excels at SQL. Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.',
   'role': 'system'},
  {'content': '### instruction:승리가 63이고 패배가 76보다 작은 경우, 승률은 얼마인가요? ### Input:CREATE TABLE table_32202 (\n    "Year" real,\n    "Team Name" text,\n    "Wins" real,\n    "Losses" real,\n    "W-L %" text,\n    "Manager" text\n) ### response:',
   'role': 'user'}],
 'SELECT "W-L %" FROM table_32202 WHERE "Wins" = \'63\' AND "Losses" < \'76\'<|im_end|>',
 'SELECT "W-L %" FROM table_32202 WHERE "Wins" = \'63\' AND "Losses" < \'76\'']

In [2]:
# ------------------------------------------------------------------------------
# 1. 1차 필터: Exact Match
#    (문자열이 완전히 동일하면 True, 아니면 False)
# ------------------------------------------------------------------------------
generated_result = [temp[1].replace("<|im_end|>", "").strip() == temp[2].replace("<|im_end|>", "").strip() for temp in result]

# Exact Match 기준으로 ACC(정확도)를 구합니다.
accuracy = sum(generated_result)/len(generated_result)
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 60.67%


In [3]:
import os
import json
from pathlib import Path
from pydantic import BaseModel
from openai import OpenAI
from pqdm.processes import pqdm


# ------------------------------------------------------------------------------
# 2. False(= 서로 다른 쿼리)인 것만 모아서 OpenAI에 의미적 비교
# ------------------------------------------------------------------------------
differing_items = []  # (index, problem, generated_sql, ground_truth_sql)
for i, is_match in enumerate(generated_result):
    if not is_match:
        problem_desc, gen_sql, gt_sql = result[i]
        differing_items.append((i, problem_desc, gen_sql, gt_sql))

print(f"[2차 OpenAI 검증] Exact Match로는 {len(differing_items)}개가 불일치로 간주되어, OpenAI로 의미 비교를 진행합니다.\n")


[2차 OpenAI 검증] Exact Match로는 2063개가 불일치로 간주되어, OpenAI로 의미 비교를 진행합니다.



/root/anaconda3/envs/text_to_sql/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# ------------------------------------------------------------------------------
# OpenAI 및 Pydantic 설정
# ------------------------------------------------------------------------------
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

client = OpenAI()

# 결과 파일 저장 폴더
save_folder = Path("/root/workspace/text_to_sql/openai_result_qwen2.5_all")
save_folder.mkdir(parents=True, exist_ok=True)

class SqlComparisonResult(BaseModel):
    # answer: 1 (의미적으로 동일) 또는 0 (의미적으로 다름)
    answer: int
    explanation: str


def one_compare_sql_semantics(item_tuple) -> dict:
    """
    item_tuple: (idx, problem_desc, gen_sql, gt_sql)
    OpenAI에 의미적 비교 요청 -> 'answer'(1/0), 'explanation' 반환
    """
    idx, problem_description, generated_query, ground_truth_query = item_tuple
    save_path = save_folder / f"result_{idx}.json"

    # 이미 결과 파일이 존재하면, 재요청하지 않고 스킵
    if save_path.exists():
        print(f"[{idx}] 이미 파일이 있습니다.")
        # 저장된 결과 불러오기
        return None

    # 프롬프트 작성
    prompt = """아래에는 한 개의 문제와 두 개의 SQL 쿼리가 주어집니다.

이때, "두 쿼리가 문자열로 달라도 실제로 동일한 결과를 반환하는지"를 판단하세요.  
구체적으로 다음 기준에 따라 판정하십시오:

1. 결과 집합(ROW 세트)이 동일한지 여부만을 판단하십시오.
- 대소문자 차이, 공백, 세미콜론 유무, 쿼리 포맷(줄바꿈 등), 작은따옴표 vs 큰따옴표, 
    테이블/컬럼 별칭 등이 다르더라도 결과 집합이 같다면 '동일'이라고 간주합니다.
2. 문법상 다른 함수나 구조를 사용했더라도 결과가 같으면 동일합니다.
- 예: GROUP BY 대상이 달라도 실제 결과가 동일하면 '동일'
        ORDER BY id vs ORDER BY season 이 실제로 같은 순서를 의미하면 '동일'
3. 만약 실제 결과가 다르다면, answer에 "0"을 적고 그 이유를 설명해 주세요.
4. 답변은 아래 JSON 형식으로만 작성해 주세요 (다른 텍스트, 문장 포함 금지):
- **"answer"**: 
  - "1"이면 두 쿼리가 의미적으로 동일한 결과를 반환한다는 의미
  - "0"이면 의미적으로 다른 결과를 반환한다는 의미  
- **"explanation"**: 
  - 왜 동일한지 또는 왜 다른지를 한글로 간단히 설명  
  - 쿼리가 완전히 같은지, 컬럼명이 달라도 동일한 컬럼을 참조하고 있는지, GROUP BY 차이가 결과에 영향을 주는지/주지 않는지, etc. 상세히 기술

### (예시) 
1. **(동일한 결과 예시)**  
   - 쿼리 A: `SELECT "Semifinalists" FROM table_31066 WHERE "Runner-up" = 'Sammy Giammalva'`  
   - 쿼리 B: `SELECT "Semifinalists" FROM table_31066 WHERE "Runner-up" = 'sammy zimmalva'`  
   - 이 경우는 **Runner-up 이름이 전혀 다르므로 실제 결과가 다르다** → `answer: 0`  
   
2. **(동일한 결과 예시)**  
   - 쿼리 A: `SELECT T1.CName FROM COURSE AS T1 JOIN ENROLLED_IN AS T2 ON T1.CID = T2.CID GROUP BY T2.CID HAVING COUNT(*) >= 5`  
   - 쿼리 B: `SELECT T1.CName FROM COURSE T1 JOIN ENROLLED_IN T2 ON T1.CID = T2.CID GROUP BY T1.CName HAVING COUNT(*) >= 5`  
   - 만약 `CName`과 `CID`가 1:1 대응되어 있어 실제 반환되는 행이 **동일**하다면 `answer=1`.  
   - 만약 CName에 중복이 있을 수 있어 결과가 달라진다면 `answer=0`.

3. **(동일한 결과 예시)**  
   - 쿼리 A: `SELECT "west" FROM table_204_1 ORDER BY "season" DESC LIMIT 1`  
   - 쿼리 B: `SELECT "west" FROM table_204_1 ORDER BY id DESC LIMIT 1`  
   - 만약 `id`와 `"season"`이 완전히 같은 순으로 증가한다면 **결과가 같다** → `answer=1`.  
   - 만약 그렇지 않다면(예: id가 엉뚱하게 매겨짐) → `answer=0`.

위 사항을 참고하여, 
- **두 SQL 쿼리가 의미적으로 동일한지** (결과 집합이 같은지)  
- 대소문자, 공백, 별칭 차이 등은 **결과에 영향을 주지 않으므로** `answer=1`로 처리  
- 실제 결과가 다른 경우만 `answer=0` 처리하고 왜 다른지 짧게 설명  

이 지침에 **엄격히** 따라 판단하십시오.

    {{
    "answer": "1 또는 0",
    "explanation": "이유를 한글로 간결히"
    }}

    위 지침을 만족하도록 문제와 쿼리 정보를 참고해 판단해 주세요.

    ----------------------------
    문제 설명:
    {problem_description}

    생성된 쿼리:
    {generated_query}

    정답(원본) 쿼리:
    {ground_truth_query}

    판단 후, 
    - 두 쿼리가 의미적으로 동일하면 "answer": "1"
    - 다르면 "answer": "0"
    그리고 "explanation"에 그 이유를 한글로 적어주세요.
    """.strip()

    try:
        # Structured Output
        completion = client.beta.chat.completions.parse(
            model="gpt-4o",  # 사용 가능한 모델로 변경
            messages=[
                {
                    "role": "system",
                    "content": (
                        "주어진 문제의 맥락에서 SQL 쿼리의 의미적 의미를 비교하는 유용한 도우미입니다. "
                        "explanation은 한글로 작성하세요."
                    )
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            response_format=SqlComparisonResult,
        )
        parsed_result = completion.choices[0].message.parsed
        
        # dict로 변환
        result_dict = {
            "index": idx,
            "answer": parsed_result.answer,
            "explanation": parsed_result.explanation
        }
        # 파일로 저장
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(result_dict, f, ensure_ascii=False, indent=4)

        # print(f"[{idx}] Saved: answer={parsed_result.answer}")
        return result_dict

    except Exception as e:
        print(f"[{idx}] Error processing: {e}")
        return {
            "index": idx,
            "answer": -1,
            "explanation": f"Error: {str(e)}"
        }


In [5]:

# ------------------------------------------------------------------------------
# 3. pqdm을 이용한 병렬 처리
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    from pqdm.processes import pqdm

    # 병렬 처리 (CPU core 개수에 맞춰 n_jobs 조정)
    results = pqdm(differing_items, one_compare_sql_semantics, n_jobs=5)

    # 2차 검증 결과 중 answer가 1인 것(의미적으로 동일하다고 판정)
    # => 몇 개나 되는지 집계
    valid_results = [r for r in results if r["answer"] in (0,1)]
    if len(valid_results) == 0:
        print("\n[OpenAI 검증] 유효 결과가 없습니다.")
    else:
        semantically_same = sum(r["answer"] == 1 for r in valid_results)
        semantically_diff = sum(r["answer"] == 0 for r in valid_results)
        print(f"\n[OpenAI 검증] 의미적으로 동일(1): {semantically_same}, "
              f"다름(0): {semantically_diff}, 총 {len(valid_results)}개")


    # 참고) 이후에는 '의미적 동일(=1)'인 케이스가 얼마나 되는지
    #      추가 통계(Overall Accuracy 등)도 계산할 수 있습니다.


QUEUEING TASKS | : 100%|██████████| 2063/2063 [00:00<00:00, 16637.06it/s]
PROCESSING TASKS | : 100%|██████████| 2063/2063 [14:23<00:00,  2.39it/s]
COLLECTING RESULTS | : 100%|██████████| 2063/2063 [00:00<00:00, 181401.45it/s]


[OpenAI 검증] 의미적으로 동일(1): 127, 다름(0): 1936, 총 2063개


In [6]:
import os
import json
from pathlib import Path

# 결과가 저장된 폴더
save_folder = Path("/root/workspace/text_to_sql/openai_result_qwen2.5_all")

# 파일 목록 가져오기
json_files = sorted(save_folder.glob("result_*.json"))

num_files = 0
num_correct = 0

for file_path in json_files:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        # data 형태 예시:
        # {
        #   "answer": 1,
        #   "explanation": "..."
        # }
        
        # answer가 1이면 "의미적으로 동일"이라고 판단한 것이므로 맞았다고 봄
        if data.get("answer") == 1:
            num_correct += 1
        num_files += 1

# 간단한 통계
if num_files > 0:
    accuracy = (num_correct / num_files) * 100
    print(f"정답 개수: {num_correct}/{num_files} 개 (정확도: {accuracy:.2f}%)")
else:
    print("결과 파일이 없습니다.")


정답 개수: 127/2063 개 (정확도: 6.16%)


In [7]:
total_accuracy = (sum(generated_result) + num_correct) / len(generated_result) * 100
print(f"정확도 : {total_accuracy:.2f}%")

정확도 : 63.09%


In [61]:
differing_items[10:200]

[(27,
  [{'content': 'You are a helpful programmer assistant that excels at SQL. Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.',
    'role': 'system'},
   {'content': '### instruction:마헬라 자야워드네와 틸란 사마라위라가 몇 점을 기록했나요? ### Input:CREATE TABLE table_name_82 (\n    runs INTEGER,\n    batting_partners VARCHAR\n) ### response:',
    'role': 'user'}],
  'SELECT SUM(runs) FROM table_name_82 WHERE batting_partners = "mahela jayawardene and thilan samaraweera"<|im_end|>',
  'SELECT SUM(runs) FROM table_name_82 WHERE batting_partners = "manjeera jayawardene and thilan samaraweera"'),
 (30,
  [{'content': 'You are a helpful programmer assistant that excels at SQL. Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.',
    'role': 'system'},
   